In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import bcolz
import pickle
import re
import math
import pandas as pd
from tqdm import tqdm
import pytorch_lightning as pl
#Dataset
from torch.utils.data import Dataset


torch.manual_seed(1)

In [ ]:
# # import pre-trained glove
# words = []
# idx = 0
# word2idx = {}

# vectors = bcolz.carray(np.zeros(1), rootdir=f'D:\Jupyter\Glove6B/6B.{dimension}.dat', mode = "w")

# with open(f'D:\Jupyter\Glove6B/glove.6B.{dimension}d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(float)
#         vectors.append(vect)

# vectors = bcolz.carray(vectors[1:].reshape((-1, 50)), rootdir=f'D:\Jupyter\Glove6B/6B.{dimension}.dat', mode = "w")
# vectors.flush()

# pickle.dump(words, open(f'D:\Jupyter\Glove6B/6B.{dimension}_words.pk1', 'wb'))
# pickle.dump(word2idx, open(f'D:\Jupyter\Glove6B/6B.{dimension}_idx.pk1', 'wb'))

# vectors = bcolz.open(f'D:\Jupyter\Glove6B/6B.{dimension}.dat')[:]
# words = pickle.load(open(f'D:\Jupyter\Glove6B/6B.{dimension}_words.pk1', 'rb'))
# word2idx = pickle.load(open(f'D:\Jupyter\Glove6B/6B.{dimension}_idx.pk1', 'rb'))

# glove = {w: vectors[word2idx[w]] for w in words}

In [8]:
# train_sentence = """When forty winters shall besiege thy brow,
# And dig deep trenches in thy beauty's field,
# Thy youth's proud livery so gazed on now,
# Will be a totter'd weed of small worth held:
# Then being asked, where all thy beauty lies,
# Where all the treasure of thy lusty days;
# To say, within thine own deep sunken eyes,
# Were an all-eating shame, and thriftless praise.
# How much more praise deserv'd thy beauty's use,
# If thou couldst answer 'This fair child of mine
# Shall sum my count, and make my old excuse,'
# Proving his beauty by succession thine!
# This were to be new made when thou art old,
# And see thy blood warm when thou feel'st it cold.""".split()

# EMBEDDING_DIM = dimension

# vocab = set(train_sentence)
# vocab_size = len(vocab)
# weights_matrix = np.zeros((vocab_size, dimension))
# words_found = 0

# for i, word in enumerate(vocab):
#     try: 
#         weights_matrix[i] = glove[word]
#         words_found += 1
#     except KeyError:
#         weights_matrix[i] = np.random.normal(scale=0.6, size = (EMBEDDING_DIM, ))

In [10]:
# def create_emb_layer(weights_matrix, non_trainable = False):
#     num_embeddings, embedding_dim = weights_matrix.size()
#     emb_layer = nn.Embedding(num_embeddings, embedding_dim)
#     emb_layer.load_state_dict({'weight': weights_matrix})
#     if non_trainable:
#         emb_layer.weight.requires_grad = False
    
#     return emb_layer, num_embeddings, embedding_dim

# class ToyNN(nn.Module):
#     def __init__(self, weights_matrix, hidden_size, num_layers):
#         super(self).__init__()
#         self.embedding, num_embeddings, embedding_dim = create_emb_labyer(weights_matrix, True)
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first = True)
        
#     def forward(self, inp, hidden):
#         return self.gru(self.embedding(inp), hidden)
    
#     def init_hidden(self, batch_size):
#         return Variable(torch.zeros(self.num_layers, batch_size, self.heiidn_size))

In [3]:
def fnv1a(txt):
    # 64 bit fnv-1a
    txt = bytes(txt, 'utf-8')
    hval = 0xcbf29ce484222325
    fnv_prime = 0x100000001b3
    for c in txt:
        hval = hval ^ c
        hval = (hval * fnv_prime) % K
    return hval        

def subword_hashes(word):
    sub_hash = []
    tword = '<' + word + '>'
    sub_hash.append(fnv1a(tword))
    for n in range(3,7):
        for i in range(len(tword)-n+1):
            sub_hash.append(fnv1a(tword[i:i+n]))
    return sub_hash
            

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import sys
import os
# from gensim.corpora import WikiCorpus
import glob
# import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [70]:
# def make_corpus(in_f, out_f):
#     output = open(out_f, 'w')
#     wiki = WikiCorpus(in_f)
#     i = 0
#     for text in wiki.get_texts():
#         output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
#         i += 1
#         if (i % 1e4 == 0):
#             print('Processed ' + str(i) + ' articles')
#     output.close()
#     print('Processing Completed!')

In [5]:
class fasttext_dataset(Dataset):
    def __init__(self, in_f, SOURCE, CONTEXT_SIZE, SUBSAMPLING, REJ_THRESHOLD, MIN_FREQ, FORMAT, limit = None):
        vocab, text_dict = self.make_dict(in_f, SOURCE, SUBSAMPLING, REJ_THRESHOLD, MIN_FREQ, FORMAT, limit)
        training_data = self.get_training_data(text_dict, CONTEXT_SIZE)
        
        self.vocab = vocab
        self.text_dict = text_dict
        
        self.data = torch.tensor(training_data, dtype = torch.long)
    
    def __getitem__(self, idx):
        target = self.data[idx, 0]
        context = self.data[idx, 1]
        return target, context
    
    def __len__(self):
        return len(self.data)
    
    def make_dict(self, in_f, source, subsampling, rej_threshold, min_freq, FORMAT, limit):
        print("Gathering texts....")
        body_tag = FORMAT[source]
        t_dict = {}
        text_list = [None]
        for path in tqdm(glob.glob(in_f)):
            if limit is not None:
                limit -= 1
                if limit < 0:
                    break
            f = open(path, errors = 'ignore')
            data = f.read()
            file_type = path[len(path) -3:]
            if file_type == 'xml':
                root = BeautifulSoup(data, 'xml')
                paragraphs = root.find_all(body_tag)
            elif file_type == 'txt':
                paragraphs = [data]
            else:
                paragraphs = [data]
                
            for x in paragraphs:
                text = x.text.lower()
                words = word_tokenize(text)
                for w in words:
                    text_list.append(w)
                text_list.append(None)
            f.close()
            
#         print("Lemmatizing and removing stopwords...")
#         lem_text_list = []
#         lem = WordNetLemmatizer()
#         stop_words = set(stopwords.words('english'))
#         stop_words = set()
        for w in tqdm(text_list):
            if w is None:
                lem_text_list.append(w)
                continue
            w = lem.lemmatize(w, 'v')
#             if w not in stop_words:
            lem_text_list.append(w)
        voc, text_list = self.get_word_freqs(lem_text_list, subsampling, rej_threshold, min_freq)
#         print("\n\n\n", vocab)
        print("Constructing Dictionary...")
        for idx, w in tqdm(enumerate(text_list)):
            t_dict[idx] = w
#         print(text_dict)
        print("Dictionary creation completed.")
        return voc, t_dict
    
    def get_training_data(self, text_dict, context_size):
        training_data = []
        for t_idx in range(len(text_dict)):
            if text_dict[t_idx] is None:
                continue
            for sign in [-1,1]:
                for w in range(1, context_size+1):
                    c_idx = t_idx + w*sign
                    if text_dict[c_idx] is None:
                        break
                    training_data.append([t_idx, c_idx])
        return training_data
    
    def get_word_freqs(self, text_list, subsampling, rej_threshold, min_freq):
        v = {}
        total = 0.0
        new_text_list = []
        for word in text_list:
            if word is not None:
                if word not in v:
                    v[word] = 0.0
                v[word] += 1.0
                total += 1.0
#         print("\nSubsampling: ", subsampling)
# #         a = 100
# #         print(len(text_list))
#         if subsampling:
#             print("Subsampling...")
#             for w in tqdm(text_list):
# #                 print(word)
#                 word = w
#                 if word is not None:
#                     fq = v[word]/total
#                     prob = 1 - np.sqrt(rej_threshold/fq)
#                     sampling = np.random.sample()
#                     print(word, prob, sampling)
#                     if sampling < prob or v[word] < min_freq:
#                         text_list.remove(word)
#             print(len(text_list))
        return v, text_list

In [6]:
in_folder = "D:/VISTEC Intern/corpus/blogs/*"
SOURCE = 'blog'
CONTEXT_SIZE = 3
SUBSAMPLING = True
REJ_THRESHOLD = 1e-5
MIN_FREQ = 2
FORMAT = {'blog':'post'}

# total embeddings
K = int(2e6)

dimension = 50
neg_samples = 5

In [7]:
train_dataset = fasttext_dataset(in_folder, SOURCE, CONTEXT_SIZE, SUBSAMPLING, REJ_THRESHOLD, 
                                 MIN_FREQ, FORMAT, limit = 10)
# print(train_dataset.text_dict)

Gathering texts....


  0%|          | 0/229307 [00:00<?, ?it/s]

Lemmatizing and removing stopwords...


  2%|▏         | 2904/136456 [00:00<00:04, 29013.81it/s]


Subsampling:  True
Subsampling...


 54%|█████▍    | 73446/136456 [00:14<00:12, 4980.68it/s] 
73446it [00:00, 2727476.66it/s]


Constructing Dictionary...
Dictionary creation completed.


In [15]:
class FastText_Model(pl.LightningModule):
    
    def __init__(self, subword_size, embedding_dim, distribution):
        super().__init__()
        self.subword_embeddings = nn.Embedding(subword_size, embedding_dim)
        self.embedding_size = subword_size
        self.distribution = distribution
        self.subword_embeddings.weight.data.uniform_(-0.05,0.05)
    
    def forward(self, x):
        print(target_idx, context_idx)
        target = train_dataset.text_dict[target_idx]
        t_hash = subword_hashes(target)
        t_vec = torch.zeros_like(self.subword_embeddings(t_hash[0]))
        for h in t_hash:
            t_vec += self.subword_embeddings(h)
            
        context = train_dataset.text_dict[context_idx]
        c_hash = subword_hashes(target)
        c_vec = torch.zeros_like(t_vec)
        for h in c_hash:
            c_vec += self.subword_embeddings(h)
        
        # negatives    
        cts = set()
        for sign in [-1,1]:
            for w in range(1, CONTEXT_SIZE+1):
                c_idx = target_idx + w*sign
                if text_dict[c_idx] is None:
                    break
                cts.add(text_dict[c_idx])
        W = {}
        P_vec = []
        for w in self.distribution:
            if w not in cts:
                total += self.distribution[w]
                
        for i,w in enumerate(self.distribution):
            if w not in cts:
                fq = self.distribution[w]/total
                prob = 1 - np.sqrt(REJ_THRESHOLD/fq)
                W[i] = w
                P_vec.append(prob)
        distr = torch.tensor(P_vec, dtype = torch.double)
        ngs = torch.multinomial(distr, neg_samples, replacement = True)
        ns = [W([int(i)]) for i in ngs]
        print(ns)
        hashes = [subword_hashes(w) for w in ns]
        ns_vec = []
        for hsh in hashes:
            vec = torch.zeros_like(t_vec)
            for h in hsh:
                vec += self.subword_embeddings(h)
            ns_vec.append(vec)
#         return t_vec, c_vec, ns_vec
        return torch.nn.Softmax()
    
    def training_step(self, batch, batch_idx)
        debug = True
    
        emb_product = torch.mul(t_vec, c_vec)
        if debug:print("positive product.shape", emb_product.shape)
            
        emb_product = torch.sum(emb_product, dim=1)
        if debug:print("positive sum.shape", emb_product.shape)
            
        pos_loss = F.logsigmoid(emb_product)
        if debug:print("pos_loss.shape", pos_loss.shape)
            
        # negative
        neg_sum = torch.zeros_like(emb_product)
        for nv in ns_vec:
            neg_product = torch.mul(t_vec, nv)
            if debug:print("negative product.shape", neg_product.shape)
            
            neg_product = torch.sum(neg_product, dim=1)
            if debug:print("negative sum.shape", neg_product.shape)
            
            neg_sum += neg_product
            if debug:print("negative sum.shape", neg_loss.shape)
        
        loss = 1
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 0.01)
    

In [39]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 20)
trainer = pl.Trainer()
model = FastText_Model(K, dimension, trainloader.vocab)

In [19]:
a = torch.tensor([[1,1,1]], dtype =  torch.long)
b = torch.tensor([[3,4,5],[1,1,1]], dtype = torch.long)
print(a.shape)
print(b.shape)
print(torch.mul(a,b))

torch.Size([1, 3])
torch.Size([2, 3])
tensor([[3, 4, 5],
        [1, 1, 1]])


In [18]:
# N_gram
context_size = 2
Cgram = []
for i in range(len(train_sentence)):
    temp_context = []
    for j in range(max(0,i - context_size+1), min(len(train_sentence),i + context_size)):
#         print(train_sentence[j], " ", train_sentence[i])
        if(j != i):
            temp_context.append(train_sentence[j])
    Cgram.append((temp_context, train_sentence[i]))
    
print(Cgram)


NameError: name 'train_sentence' is not defined

In [151]:
class FastText_neg_sampling(nn.Module):
    
    def __init__(self, vocab_size, subtext_size, embedding_dim, context_size, noise_dist = None, negative_samples = 10):
        super(FastText_neg_sampling, self).__init__()
#         self.embeddings_input = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings_subtext = nn.Embedding(subtext_size, embedding_dim)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_dim)
        self.vocab_size = vocab_size
        self.negative_samples = negative_samples
        self.noise_dist = noise_dist
        
#         self.embeddings_input.weight.data.uniform_(-1,1)
        self.embeddings_context.weight.data.uniform_(-0.05,0.05)
        self.embeddings_subtext.weight.data.uniform_(-0.05,0.05)
        
#         self.linear1 = nn.Linear(context_size*embedding_dim, 128)
#         self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, subtext, context_word):
        debug = True
        if debug:
#             print('input_word.shape: ', input_word.shape)
            print('context_word.shape: ', context_word.shape)
            print('subtext_word.shape: ', subtext.shape)
            
#         emb_input = self.embeddings_input(input_word)
#         if debug: print('emb_input.shape: ', emb_input.shape)
        
        emb_context = self.embeddings_context(context_word)  
        if debug:print('emb_context.shape: ', emb_context.shape)
        
        emb_target = self.embeddings_subtext(subtext)
        emb_target = torch.sum(emb_target, 0).view(1,-1)
        if debug:print('emb_subtext.shape: ', emb_target.shape)
        
        emb_product = torch.mul(emb_target, emb_context)
        if debug: print('emb_product.shape: ', emb_product.shape)
        
        emb_score = torch.sum(emb_product, dim=1)
        if debug: print('emb_score.shape: ', emb_score.shape)

        out_loss = F.logsigmoid(emb_score)   
#         out_loss = math.log(1 - math.exp(-emb_product))
        if debug:print('out_loss.shape: ', out_loss.shape)
            
################ NEGATIVE SAMPLING #####################

        if self.noise_dist is None:
            noise_dist = torch.ones(self.vocab_size)
        else:
            noise_dist = self.noise_dist

        if debug: print('noise_dist.shape: ', noise_dist.shape)

        batch_neg_samples_num = context_word.shape[0]*self.negative_samples
        negative_example = torch.multinomial(noise_dist, batch_neg_samples_num, replacement = True)
        if debug: print('negative_example.shape: ', negative_example.shape)

        negative_example = negative_example.view(context_word.shape[0], self.negative_samples) # bs, num_neg_samples
        if debug:print('negative_example.shape: ', negative_example.shape)

        emb_negative = self.embeddings_context(negative_example) # bs, neg_samples, emb_dim
        if debug:print('emb_negative.shape: ', emb_negative.shape)

        if debug: print('emb_target.unsqueeze(2).shape: ', emb_target.unsqueeze(2).shape)
        emb_product_neg_samples = torch.bmm(emb_negative.neg(), emb_target.unsqueeze(2))
        if debug: print('emb_product_neg_samples.shape: ', emb_product_neg_samples.shape)

        noise_loss = F.logsigmoid(emb_product_neg_samples).squeeze(2).sum(1)
        if debug: print('noise_loss.shape: ', noise_loss.shape)

        total_loss = -(out_loss + noise_loss).mean()
        if debug: print('total_loss.shape', total_loss.shape)

        return total_loss

In [152]:
EMBEDDING_DIM = dimension
ngs = 5

vocab = set(train_sentence)
word_to_ix = {word: index for index,word in enumerate(vocab)}
ix_to_word = {index: word for index,word in enumerate(vocab)}
subtext_to_ix = {word: index for index,word in enumerate(subtext)}
ix_to_subtext = {index: word for index,word in enumerate(subtext)}

word_count = {}
for i in vocab:
    word_count[i] = train_sentence.count(i)
# print(word_count)

word_freq = np.array([word_count[word] for word in vocab])
unigram_dist = word_freq/word_freq.sum()
ws = 1 - np.sqrt(10e-4/unigram_dist)
ws = np.clip(ws,0,1)
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

losses = []

# train_loader = torch.utils.data.DataLoader()
model = FastText_neg_sampling(len(vocab), subtext_size,  EMBEDDING_DIM, context_size, noise_dist, ngs)
optimizer = optim.SGD(model.parameters(), lr = 0.001)


EPOCH = 5

model.train()
for epoch in range(EPOCH):
    print(f'\n========== EPOCH {epoch+1}/{EPOCH} ==========')
    total_loss = 0
    for context, target in Cgram:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype = torch.long)
#         target_idx = torch.tensor([word_to_ix[target]], dtype = torch.long)
        subtext_idxs = torch.tensor([subtext_to_ix[w] for w in ngram[target]], dtype = torch.long)
        optimizer.zero_grad()
        loss = model(subtext_idxs, context_idxs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

            


========== EPOCH 1/5 ==========
context_word.shape:  torch.Size([1])
subtext_word.shape:  torch.Size([10])
emb_context.shape:  torch.Size([1, 50])
emb_subtext.shape:  torch.Size([1, 50])
emb_product.shape:  torch.Size([1, 50])
emb_score.shape:  torch.Size([1])
out_loss.shape:  torch.Size([1])
noise_dist.shape:  torch.Size([97])
negative_example.shape:  torch.Size([5])
negative_example.shape:  torch.Size([1, 5])
emb_negative.shape:  torch.Size([1, 5, 50])
emb_target.unsqueeze(2).shape:  torch.Size([1, 50, 1])
emb_product_neg_samples.shape:  torch.Size([1, 5, 1])
noise_loss.shape:  torch.Size([1])
total_loss.shape torch.Size([])
context_word.shape:  torch.Size([2])
subtext_word.shape:  torch.Size([15])
emb_context.shape:  torch.Size([2, 50])
emb_subtext.shape:  torch.Size([1, 50])
emb_product.shape:  torch.Size([2, 50])
emb_score.shape:  torch.Size([2])
out_loss.shape:  torch.Size([2])
noise_dist.shape:  torch.Size([97])
negative_example.shape:  torch.Size([10])
negative_example.shape: 

RuntimeError: Expected batch2_sizes[0] == bs && batch2_sizes[1] == contraction_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [96]:
a = {'a':1, 'b':2, 'c':20}
b = [word for word in a]
print(b)

['a', 'b', 'c']


In [14]:
a = {1,2,3}
a.add(1)
print(a)

{1, 2, 3}


In [131]:
a = torch.tensor([[1,0,0,0],[2,0,0,0],[3,0,0,0],[4,0,0,0]])
v = torch.tensor([[0,0,0,1],[0,0,0,2],[0,0,0,3],[0,0,0,4]])
xc = torch.tensor([[1,2,1,1]])
sv = torch.tensor([[1,0,0,0],[1,2,0,0]])
print(a.shape)
print(torch.sum(a, 0).view(1,-1))
print(xc.shape)
sc = torch.mul(xc,sv)
print(sc.shape, sc, sep ='\n')
s = torch.sum(sc, dim=1)
print(s.shape, s, sep = '\n')
z = a + v
print(z.shape)
print(z)
# print(torch.sum(z, dim =1))
# print(a)
# print(a.shape)
# g = torch.mul(xc,a)
# print(g)
# print(g.shape)
# print(torch.sum(a,0).view(1,-1))
# if(0): print('q')
# c = torch.tensor([[0,0,0,1] for i in range(7)])
# b = torch.tensor([], dtype = torch.long)
# print(torch.cat((-xc,-xc*(-1))))

torch.Size([4, 4])
tensor([[10,  0,  0,  0]])
torch.Size([1, 4])
torch.Size([2, 4])
tensor([[1, 0, 0, 0],
        [1, 4, 0, 0]])
torch.Size([2])
tensor([1, 5])
torch.Size([4, 4])
tensor([[1, 0, 0, 1],
        [2, 0, 0, 2],
        [3, 0, 0, 3],
        [4, 0, 0, 4]])


In [108]:
def sc(a = 1):
    print(a)
    
sc(a = 3)

3


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
print(stopwords.words('english'))

NameError: name 'stopwords' is not defined